In [1]:
from PyPDF2 import PdfReader

reader = PdfReader("./aerolineas_flight_ticket.pdf")

document = ''.join(page.extract_text() for page in reader.pages)

document

'Información Del Pasajero\nNÚMERO DE BOLETO\n0442155218038\nAPELLIDO DEL PASAJERO\nS\nI\nL\nV\nA\n/\nR\nO\nD\nR\nI\nGO\nREFERENCIA DE NOMBRE\nADT\nFECHA DE EMISIÓN\n02 SEP 2023\nAEROLÍNEA EMISORA\nAEROLINEAS ARGENTINAS\nAGENTE EMISOR\nAEROLINEAS\nARGENTINAS/WED\nUBICACIÓN DEL AGENTE\nEMISOR\nVENTAS AR, ARGENTINA\nCódigo De Reserva\nOJLPBI\nDetalles Del Itinerario\nVUELO\nSALIDA\nARRIBO\nCABINA/ASIENTO\nEQUIPAJE\n(PC =\nPIEZAS)\nINFORMACIÓN DE\nVUELO\nAR\n \n7709\n \nVALIDO\nPARA\nVIAJAR\nSAO PAULO GUARULH,\nBRAZIL \n(GRU)\n \nTERMINAL 2 \n05/sep/2023\n \n09:10\nAEROPARQUE JORGE\nNEWBERY (AEP)\n \n05/sep/2023\n \n12:10\nRequiere check-in\n1 pieza\nOperado por:\nGOL LINHAS\nAEREAS\nCódigo de\nreservación de la\naerolínea:\n \nICBRXF\nTipo de tarifa:\n \nBase de tarifa:\nKLOWBXEP\nNo válido antes del:\n05 sep\nNo válido después\ndel:\n \n05 sep\nEsta no es una tarjeta de embarque\nAR\n \n1258\n \nVALIDO\nPARA\nVIAJAR\nAEROPARQUE JORGE\nNEWBERY \n(AEP)\n \n08/sep/2023\n \n16:10\nSAO PAULO 

In [2]:
import sys
sys.path.insert(0, '../src/core/')

from llm import ILLM, Ollama, LLMBuilder                                        # type: ignore

illm: ILLM = LLMBuilder(
    llm_type='ollama',
    # model_name='llama3',
    # model_name='llama3.2:1b',
    model_name='phi3:instruct',
    temperature=0
)
ollm: Ollama = illm.get_instance()

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage                 # type: ignore

In [4]:
system_message = SystemMessage(
    content="""
      You are an assistant specialized in language translation.
      Translate the text to English to facilitate the extraction process. 
    """
)
human_message=HumanMessage( content=document )
result = ollm.invoke([system_message, human_message])
result

'Estimado cliente de Aerolíneas Argentinas Sucursal en Colombia:\n\nLe escribo para informarle sobre la reserva que ha realizado con nuestro sistema, especificando el estado actual y los detalles pertinentes. Su viaje se encuentra confirmado bajo las siguientes condiciones:\n\n- Estado de Reserva: CONFIRMADO (E)\n- Codigo Fiscale Argentina Sucursal en Italia: 00428790588 / Partita IVA: 0 end of document.'

In [5]:
system_message = SystemMessage( 
    content="""
      You are an assistant specialized in creating highly detailed descriptions based on user-provided information.
      Your task is to generate an extensive, comprehensive, and thorough description of the data given, leaving no detail unaddressed. Break down the information clearly, covering all relevant aspects with precision.
      Ensure the description is well-organized, with clear sections where appropriate. Be mindful to elaborate on context, specific elements, and nuances in the information, so the user can fully understand each point.
    """
)
human_message=HumanMessage( content=result )
result = ollm.invoke([system_message, human_message])
result

'Estimado cliente,\n\nLe escribo para informarle sobre la reserva que ha realizado con nuestro sistema de Aerolíneas Argentinas Sucursal en Colombia y proporcionarle los detalles pertinentes acerca del estado actual y las condiciones especificadas. Su viaje se encuentra confirmado bajo el siguiente código:\n\n**Estado de Reserva: CONFIRMADO (E)**\nLa reserva que ha realizado con nuestro sistema está en un estado confirmado, lo cual significa que todos los requisitos y detalles necesarios para su viaje han sido completados satisfactoriamente. Esto incluye la confirmación de asientos disponibles, el pago total del costo completo del vuelo (incluyendo impuestos), tarifas adicionales si aplicables, así como cualquier cargo extra que pueda haberse acordado previamente con nuestro personal.\n\n**Codigo Fiscale Argentina Sucursal en Italia: 00428790588 / Partita IVA: 0 end of document.**\nEl cliente ha proporcionado su código fiscal argentino para la sucursal de Aerolíneas Argentinas en Colom

In [6]:
system_message = SystemMessage( 
    content="""
      You are an assistant specialized in answering detailed questions about flight information. Based on the provided text, answer the following questions:

      1. What is the name of the departure airport?
      2. What is the IATA code for the departure airport?
      3. What is the name of the destination airport?
      4. What is the IATA code for the destination airport?
      5. What is the scheduled departure time of the flight?
      6. What is the scheduled arrival time of the flight?
      7. What is the reservation code for the flight?
      8. What are the flight dates?
      9. What are the baggage policies or information for the flight?

      Answer each question clearly and precisely, based on the details in the text.
    """
)
human_message=HumanMessage( content=result )
result = ollm.invoke([system_message, human_message])
result

'1. The departure airport is not explicitly mentioned in the provided text; therefore, I cannot provide its name based on this information alone. Additional details would be required to answer this question accurately.\n2. Similarly, there\'s no mention of an IATA code for the departure airport within the given document. To find it out, one might need more context or specific flight data that includes such codes.\n3. The destination airport is also not named in the text provided above; hence its name cannot be determined from this information alone either. Additional details would again be necessary to answer accurately.\n4. As with departure and arrival cities, there\'s no IATA code for the destination airport mentioned within the document given here. More context or specific flight data is needed to find out such codes.\n5. The scheduled departure time of the flight isn\'t provided in this text snippet; therefore it cannot be determined from the information available above. Additiona

In [7]:
system_message = SystemMessage( 
    content="""
      You are an assistant specialized in extracting flight information from text. Based on the provided text, answer the following questions and respond exclusively with a filled JSON, without any additional comments or explanations:

      {
        "departure_airport": "Answer to question 1",
        "departure_airport_code": "Answer to question 2",
        "destination_airport": "Answer to question 3",
        "destination_airport_code": "Answer to question 4",
        "departure_time": "Answer to question 5",
        "arrival_time": "Answer to question 6",
        "reservation_code": "Answer to question 7",
        "flight_dates": "Answer to question 8",
        "baggage_information": "Answer to question 9"
      }

      Only respond with the completed JSON format.
    """
)
human_message=HumanMessage( content=result )
result = ollm.invoke([system_message, human_message])
result


'```json\n{\n    "departure_airport": "",\n    "departure_airport_code": "",\n    "destination_airport": "",\n    "destination_airport_code": "",\n    "departure_time": "",\n    "arrival_time": "",\n    "reservation_code": "00428790588 / Partita IVA: 0",\n    "flight_dates": "",\n    "baggage_information": ""\n}\n```'